# Declarations

## Imports

In [ ]:
import os
import re
import math
import string
import random
import requests
import importlib
import itertools

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import plotly.graph_objects as go

from tqdm import tqdm

from IPython.display import display

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

tfk = tf.keras
tfkl = tf.keras.layers
kb = tf.keras.backend

In [ ]:
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Constants

In [ ]:
# Randomness
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
# Filepaths
kaggle = False

model_versions = ["v2.1", "v2.1"]

github_repo = "raul-singh/Rise-of-Transformers-Project"
github_branch = "main"
github_python_prefix = ["Code", "Notebooks", "py_files"]
github_clip_models_prefix = ["Code", "Models"]
github_pyfiles_data = [
    {"name": "preprocessing", "imports": ["import_datasets"]}, 
    {"name": "evaluation", "imports": [
        "EvalMetrics as evm", "generate_dataset_reference", "compute_total_relevance", "generate_image_embeddings", "generate_text_embeddings", 
        "find_t2i_matches", "find_i2t_matches", "index_to_reference", "compute_relevant_at_k"
    ]}, 
    {"name": "visualization", "imports": ["retrieval_report", "retrieval_graph_compare"]}, 
    {"name": "clip", "imports": ["build_clip"]}
]
github_pyfiles = ["/".join(github_python_prefix) + "/" + pf["name"] + ".py" for pf in github_pyfiles_data]
github_clip_models = [f"{'/'.join(github_clip_models_prefix)}/{version}.yaml" for version in model_versions]

kaggle_dataset1 = "/kaggle/input/transformers-hackathon/"
kaggle_dataset2 = "/kaggle/input/transformers-hackathon-features/"
kaggle_weights = "/kaggle/input/clip-weights/"
kaggle_relevance = "/kaggle/input/clip-relevance/"

image_dir = "./resized_train"
relevance_dir = "./relevance"
caption_pred_file = "caption_prediction_train.csv"
concept_det_file = "concept_detection_train.csv"
concept_file = "concepts.csv"
clip_weights_files = [f"{version}.h5" for version in model_versions]

if kaggle:
    image_dir = kaggle_dataset1 + image_dir
    relevance_dir = kaggle_relevance + relevance_dir
    caption_pred_file = kaggle_dataset2 + caption_pred_file
    concept_det_file = kaggle_dataset2 + concept_det_file
    concept_file = kaggle_dataset2 + concept_file
    clip_weights_files = [kaggle_weights + weight for weight in clip_weights_files]

In [ ]:
# Train/Val/Test split and filter percentages
test_size = 0.2
val_size = 0
filter_percent_dataset = 1

# Batch size
batch_size = 32

# Import dataset types and shapes
in_feat_typ = {'caption': tf.string, 'concepts': tf.bool, 'image path': tf.string}
feature_shapes = {'image': (128, 128, 3), 'caption': (), 'concepts': (8374)}

# Output dataset structure
x_features_eval = ['image path', 'image']
y_features_eval = ['caption', 'concepts']

# Define parameters for dataset import
dataset_parameters = [{
    'x_features': x_features_eval, 'y_features': y_features_eval,
    'x_dict': True, 'y_dict': True,           
    'shuffle_buffer_size': 1,
    'batch_size': batch_size,
    'cached': True,
}]

## Meta-Imports

In [ ]:
def clean_recursive_imports(source, import_list, prefix):
    import_prefix = re.sub(r"/", ".", prefix)
    for target_import in import_list:
        source = re.sub(r"from[ \t]+" + re.escape(target_import) + r"[ \t]+import", f"from {import_prefix + target_import} import", source)
    return source
    
def import_py_from_repo(repository, branch, filepath, prefix, recursive_imports_list=None):
    # Build path for retrieval and write name
    path_pre = "https://raw.githubusercontent.com/"
    path = path_pre + repository + "/" + branch + "/" + filepath 
    write_path = prefix + filepath.split("/")[-1]
    print("Downloading file from " + path)
    # Obtain raw text from file
    text = requests.get(path).text
    # Clean recursive imports
    text = clean_recursive_imports(text, recursive_imports_list, prefix) if recursive_imports_list else text
    # Create subdirectories if not exist
    os.makedirs(os.path.dirname(write_path), exist_ok=True)
    # Write file
    f = open(write_path, "w")
    f.write(text)
    f.close()

In [ ]:
if kaggle:
    for pf_data, py_file in zip(github_pyfiles_data, github_pyfiles):
        import_py_from_repo(
            github_repo, github_branch, py_file, 
            "/".join(github_python_prefix) + "/", 
            recursive_imports_list=[pf["name"] for pf in github_pyfiles_data],
        )
        import_string = f'from {".".join(github_python_prefix) + "." + pf_data["name"]} import ' + " ".join(pf_data["imports"])
        exec(import_string)
    
    for model in github_clip_models:
        import_py_from_repo(github_repo, github_branch, model, "/".join(github_clip_models_prefix) + "/")
        
else:
    from pyfiles.clip import build_clip
    from pyfiles.preprocessing import import_datasets

# Preprocessing

In [ ]:
concept_info, datasets, dataset_sizes = import_datasets(
    image_dir, caption_pred_file, concept_file, concept_det_file,
    in_feat_typ, feature_shapes,
    dataset_parameters,
    filter_percent_dataset,
    test_size, val_size,
    seed,
)

In [ ]:
# Select loaded datasets and variables
concept_list, concepts_onehot = concept_info
_, _, test_dataset = datasets[0]
train_ds_size, val_ds_size, test_ds_size = dataset_sizes

# Model Import

In [ ]:
models = []
for structure, weights, version in zip(github_clip_models, clip_weights_files, model_versions):
    print(f"Creating model {version} fom {structure}")
    clip_image_encoder, clip_text_encoder, clip = build_clip(structure, weights_path=weights)
    models.append({
        "name": version,
        "image_encoder": clip_image_encoder,
        "text_encoder": clip_text_encoder,
        "clip": clip,
    })

# Model Evaluation

## Evaluation Variables

In [ ]:
# Top-k number
k = 10
# Threshold for concept overlap metric
concept_overlap_threshold = 2
# Visualization decimal precision
decimal_precision = 4
# Index to choose model from the array of models
model_index = 0
# Dictionaries used to load/save total relevance files
relevance_fileinfo_cap = {"path": relevance_dir, "test_split": test_size, "val_split": val_size, "metric": "cap"}
relevance_fileinfo_con = {"path": relevance_dir, "test_split": test_size, "val_split": val_size, "metric": "con", "other": [("conthresh", concept_overlap_threshold)]}
# Function to preprocess data when we want to evaluate captions
reference_preprocess_cap = lambda x: x["caption"].numpy().decode('UTF-8')          
# Function to preprocess data when we want to evaluate concepts
reference_preprocess_con = lambda x: x["concepts"].numpy()
reference_preprocess_con_hash = lambda x: frozenset(sorted(np.where(x["concepts"].numpy())[0]))
# Function to compute if a match is relevant given concept arrays 
concept_relevance = lambda m, o: np.count_nonzero(np.logical_and(m, o)) >= min(concept_overlap_threshold, np.count_nonzero(m), np.count_nonzero(o))
concept_relevance_hash = lambda m, o: len(m.intersection(o)) >= min(concept_overlap_threshold, len(m), len(o))

In [ ]:
# Metric IDs
METRIC_ACCURACY = "Accuracy"
METRIC_MAP = "MAP"
METRIC_MAR = "MAR"
METRIC_F1 = "F1"

# Metric visualization parameters
metrics = [
    {"id": METRIC_ACCURACY, "name": "Accuracy", "color": "green"},
    {"id": METRIC_MAP, "name": "Mean Average Precision", "color": "blue"},
    {"id": METRIC_MAR, "name": "Mean Average Recall", "color": "red"},
    {"id": METRIC_F1, "name": "F1 Score", "color": "blueviolet"}
]

## Dataset Metrics

In [ ]:
# Dataset reference initialization
dataset_reference = generate_dataset_reference(test_dataset, lambda x, y: y | {'image path': x['image path']})

# Compute relevance for all the test queries in the dataset
tot_relevant_cap = compute_total_relevance(dataset_reference, reference_preprocess=reference_preprocess_cap, save_to_file=False, fileinfo=relevance_fileinfo_cap | {"split": "test"})
tot_relevant_con = compute_total_relevance(dataset_reference, reference_preprocess=reference_preprocess_con_hash, relevance=concept_relevance_hash, save_to_file=False, fileinfo=relevance_fileinfo_con | {"split": "test"})

# Define model comparison labels
model_labels = [{"id": model["name"] + f"({str(id(model['clip']))})", "label": model["name"]} for model in models] 

## Text to Image Task

In [ ]:
results = []
for model in models:
    print(f"\n### Scoring test data for {model['name']} ###")
    test_image_embeddings = generate_image_embeddings(
        model["image_encoder"],
        test_dataset,
        dataset_pred_map=lambda x, y: x['image'],
    )
    test_queries = test_dataset.map(lambda x, y: y["caption"])
    # Compute matching results and extrapolate relevant matches based on different criterions
    test_raw_results = find_t2i_matches(test_queries, model["text_encoder"], test_image_embeddings, k=k, normalize=True)
    test_results = index_to_reference(test_raw_results, dataset_reference)
    test_relevant_cap = compute_relevant_at_k(test_results, dataset_reference, k=k, reference_preprocess=reference_preprocess_cap)
    test_relevant_con = compute_relevant_at_k(test_results, dataset_reference, k=k, reference_preprocess=reference_preprocess_con, relevance=concept_relevance)
    
    results.append({
        "results": test_results,
        "relevant_cap": test_relevant_cap,
        "relevant_con": test_relevant_con,
    })

#### Caption equality relevance metric

In [ ]:
for model in results:
    _ = retrieval_report(
        model["results"], dataset_reference, model["relevant_cap"], tot_relevant_cap,
        k=k,
        metrics=metrics,
        title=f"Test Data - Caption equality metrics @ k={k}",
        decimal_precision=decimal_precision
    )

In [ ]:
_ = retrieval_graph_compare(
    [model["results"] for model in results], dataset_reference, model_labels, tot_relevant_cap,
    k_range=(1, k),
    metrics=metrics, 
    titlexyf=("k", None, "Test Data - Caption Equality model comparison"),
    reference_preprocess=reference_preprocess_cap
)

#### Concept overlap relevance metric

In [ ]:
for model in results:
    _ = retrieval_report(
        model["results"], dataset_reference, model["relevant_con"], tot_relevant_con,
        k=k,
        metrics=metrics,
        title=f"Test Data - Concept overlap metrics @ k={k}",
        decimal_precision=decimal_precision
    )

In [ ]:
_ = retrieval_graph_compare(
    [model["results"] for model in results], dataset_reference, model_labels, tot_relevant_con,
    k_range=(1, k),
    metrics=metrics, 
    titlexyf=("k", None, "Test Data - Concept Overlap model comparison"),
    relevance=concept_relevance,
    reference_preprocess=reference_preprocess_con
)

## Image to Text

In [ ]:
results = []
for model in models:
    print(f"\n### Scoring test data for {model['name']} ###")
    test_text_embeddings = generate_text_embeddings(
        model["text_encoder"],
        test_dataset,
        dataset_pred_map=lambda x, y: y['caption'],
    )
    test_queries = test_dataset.map(lambda x, y: x["image"])
    # Compute matching results and extrapolate relevant matches based on different criterions
    test_raw_results = find_i2t_matches(test_queries, model["image_encoder"], test_text_embeddings, k=k, normalize=True)
    test_results = index_to_reference(test_raw_results, dataset_reference)
    test_relevant_cap = compute_relevant_at_k(test_results, dataset_reference, k=k, reference_preprocess=reference_preprocess_cap)
    test_relevant_con = compute_relevant_at_k(test_results, dataset_reference, k=k, reference_preprocess=reference_preprocess_con, relevance=concept_relevance)
    
    results.append({
        "results": test_results,
        "relevant_cap": test_relevant_cap,
        "relevant_con": test_relevant_con,
    })

#### Caption equality relevance metric

In [ ]:
for model in results:
    _ = retrieval_report(
        model["results"], dataset_reference, model["relevant_cap"], tot_relevant_cap,
        k=k,
        metrics=metrics,
        title=f"Test Data - Caption equality metrics @ k={k}",
        decimal_precision=decimal_precision
    )

In [ ]:
_ = retrieval_graph_compare(
    [model["results"] for model in results], dataset_reference, model_labels, tot_relevant_cap,
    k_range=(1, k),
    metrics=metrics, 
    titlexyf=("k", None, "Test Data - Caption Equality model comparison"),
    reference_preprocess=reference_preprocess_cap
)

#### Concept overlap relevance metric

In [ ]:
for model in results:
    _ = retrieval_report(
        model["results"], dataset_reference, model["relevant_con"], tot_relevant_con,
        k=k,
        metrics=metrics,
        title=f"Test Data - Concept overlap metrics @ k={k}",
        decimal_precision=decimal_precision
    )

In [ ]:
_ = retrieval_graph_compare(
    [model["results"] for model in results], dataset_reference, model_labels, tot_relevant_con,
    k_range=(1, k),
    metrics=metrics, 
    titlexyf=("k", None, "Test Data - Concept Overlap model comparison"),
    relevance=concept_relevance,
    reference_preprocess=reference_preprocess_con
)